In [ ]:
# one-time: PEFT + bitsandbytes + trl
!pip install -q peft==0.10.0 trl==0.8.6 bitsandbytes==0.45.5 transformers==4.51.3 accelerate==1.7.0 datasets==2.20.0 sqlalchemy==2.0.30 sqlite-utils==3.38 evaluate==0.4.3 torch==2.6.0

In [ ]:
!pip list | grep -E 'peft|trl|triton|bitsandbytes|transformers|accelerate|datasets|sqlalchemy|sqlite-utils|moz-sql-parser|evaluate|torch'

accelerate                            1.7.0
bitsandbytes                          0.45.5
datasets                              2.20.0
evaluate                              0.4.3
fastrlock                             0.8.3
peft                                  0.10.0
sentence-transformers                 4.1.0
sqlite-utils                          3.38
tensorflow-datasets                   4.9.8
transformers                          4.51.3
triton                                3.2.0
trl                                   0.8.6
vega-datasets                         0.9.0


In [ ]:
%%bash
python - <<'PY'
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
print("Seeds fixed to", SEED)
PY


Seeds fixed to 42


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

login()

In [ ]:
from datasets import load_dataset

spider_train   = load_dataset("spider")["train"]
spider_val = load_dataset("spider")["validation"]

print(spider_train)
print(spider_val)

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})
Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-3.1-8B"  # use preferred model
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
!pip install -q kagglehub            # tiny, pure-python package
import kagglehub, os, json, zipfile
from pathlib import Path

# Either:
#   kagglehub.login()                              # will prompt
#   – or –
os.environ["KAGGLE_USERNAME"] = ""        # <- set once per session
os.environ["KAGGLE_KEY"]     = ""

In [ ]:
import kagglehub, zipfile
from pathlib import Path

root = Path( kagglehub.dataset_download(
           "jeromeblanchet/yale-universitys-spider-10-nlp-dataset") )

# 1️⃣  Look for a zip first (old mirror versions)
zip_files = list(root.rglob("spider.zip"))
if zip_files:
    print("Found spider.zip – extracting…")
    with zipfile.ZipFile(zip_files[0], "r") as zf:
        zf.extractall("spider")          # creates ./spider/database/…
    SPIDER_DIR = Path("spider")

# 2️⃣  Otherwise assume the folder is already there (current mirror)
else:
    print("No spider.zip – using unpacked folder")
    # the mirror root itself is usually “…/spider/”, but we search just in case
    try:
        SPIDER_DIR = next(root.rglob("spider/database")).parent
    except StopIteration as e:
        raise FileNotFoundError(
            "Could not locate Spider data inside the mirror download") from e

DB_ROOT = SPIDER_DIR / "database"
print("Spider databases live at:", DB_ROOT.resolve())


No spider.zip – using unpacked folder
Spider databases live at: /root/.cache/kagglehub/datasets/jeromeblanchet/yale-universitys-spider-10-nlp-dataset/versions/1/spider/database


In [ ]:
DB_ROOT = Path("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database")

In [ ]:
#   reflect every .sqlite file to create a plain-text schema description


import sqlalchemy as sa

_schema_cache: dict[str, str] = {}            # {db_id: "…schema string…"}

def get_schema_string(db_id: str) -> str:
    """Return a compact textual schema for the given Spider database."""
    if db_id in _schema_cache:
        return _schema_cache[db_id]

    db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
    engine  = sa.create_engine(f"sqlite:///{db_file}")
    insp    = sa.inspect(engine)

    parts   = []
    for tbl in sorted(insp.get_table_names()):
        cols = [c["name"] for c in insp.get_columns(tbl)]
        parts.append(f"{tbl}({', '.join(cols)})")

    schema_str = ", ".join(parts)
    _schema_cache[db_id] = schema_str
    return schema_str

In [ ]:
from datasets import load_dataset
import random

# Use any split you like – here we sample 5 from Spider-train
NUM_SHOTS      = 5
spider_train   = load_dataset("spider")["train"]

random.seed(42
            )
# Turn each row into a normal dict and attach the schema string
DEMO_SET = [
    {
        **ex,                                  # copy all original fields
        "schema_str": get_schema_string(ex["db_id"])
    }
    for ex in spider_train.shuffle(seed=42).select(range(NUM_SHOTS))
]


In [ ]:
from datasets import load_dataset, DatasetDict

spider_train = load_dataset("spider")["train"]
half_size    = len(spider_train) // 5                      # 20 %
train_subset = spider_train.shuffle(seed=42).select(range(half_size))

def make_prompt(example):
    schema = get_schema_string(example["db_id"])
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given a database schema and a natural-language\n"
        "question, write ONE syntactically correct SQL query that answers the question.\n"
        "Return **only** the SQL; do not repeat the schema or add explanations.\n\n"

        f"### Database Schema:\n{schema}\n\n"

        f"### Question:\n{example['question']}\n\n"

        "### SQL:\n"
        f"{example['query'].strip()}"
    )

train_subset = train_subset.map(lambda ex: {"text": make_prompt(ex)}) \
                           .remove_columns(set(train_subset.column_names) - {"text"})

# sanity:
print(train_subset[0]["text"][:400], "...")
print("Fine-tune set size:", len(train_subset))


### Instruction:
You are an expert SQL developer. Given a database schema and a natural-language
question, write ONE syntactically correct SQL query that answers the question.
Return **only** the SQL; do not repeat the schema or add explanations.

### Database Schema:
player(Player_ID, Player, Team, Age, Position, School_ID), school(School_ID, School, Location, Enrollment, Founded, Denomination, B ...
Fine-tune set size: 1400


In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit               = True,
    bnb_4bit_use_double_quant  = True,
    bnb_4bit_quant_type        = "nf4",
    bnb_4bit_compute_dtype     = torch.bfloat16,   # bf16 saves VRAM vs fp16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_cfg,
    device_map="auto",
)
tokenizer.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    r                = 64,
    lora_alpha       = 128,
    target_modules   = ["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout     = 0.05,
    bias             = "none",
    task_type        = TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(base_model, lora_cfg)

# ↓ this halves activation memory at the cost of ~15 % speed
peft_model.gradient_checkpointing_enable()
peft_model.enable_input_require_grads()

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer           # same as before

args = TrainingArguments(
    output_dir                  = "/content/llama3_8b_spider_qlora",
    per_device_train_batch_size = 2,          # ↓ was 4
    gradient_accumulation_steps = 8,          # effective batch still 16
    num_train_epochs            = 5,          # your new request
    learning_rate               = 2e-4,
    fp16                        = False,      # bfloat16 already set
    bf16                        = True,
    logging_steps               = 50,
    save_strategy               = "epoch",
    save_total_limit            = 2,
    seed                        = 42,
)

trainer = SFTTrainer(
    model              = peft_model,
    tokenizer          = tokenizer,
    train_dataset      = train_subset,
    dataset_text_field = "text",
    args               = args,
    max_seq_length     = 2048,
)

trainer.train()
trainer.save_model(args.output_dir)     # LoRA adapters + config only (~350 MB)


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,0.728700
100,0.273200
150,0.193000


In [ ]:
train_subset

Dataset({
    features: ['text'],
    num_rows: 1400
})

In [ ]:
import torch, gc

# 1️⃣  free trainer / old models
del trainer, peft_model, base_model
gc.collect(); torch.cuda.empty_cache()

# 2️⃣  reload quantised base model (4-bit)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_cfg.base_model_name_or_path,
    quantization_config=bnb_cfg,
    device_map={"": 0},          # ← keep all layers on GPU 0
    torch_dtype=torch.bfloat16,
)

# 3️⃣  attach LoRA adapters
ft_model = PeftModel.from_pretrained(base_model, output_dir)  # no device_map needed
ft_model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit

# Introducing zero-shot prompting to the Qlora Adapters

In [ ]:
def build_prompt(nl_question: str, target_schema: str) -> str:
    """
    Zero-shot prompt for the fine-tuned QLoRA adapters of Llama 3.1.

    Layout:
      1.  Instruction block            – tells the model to output ONE query only
      2.  Database schema (Spider)     – table and column names
      3.  Natural-language question
      4.  Sentinel “### SQL:”          – model must continue with a single SQL
    """
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given the database schema and the "
        "natural-language question below, write ONE syntactically correct SQL "
        "statement that answers the question. Output *only* the SQL; do not "
        "repeat the schema or add explanations.\n\n"

        f"### Database Schema:\n{target_schema}\n\n"

        f"### Question:\n{nl_question}\n\n"

        "### SQL:\n"
    )

In [ ]:
import re

def generate_sql(nl_question: str, schema: str) -> str:
    prompt     = build_prompt(nl_question, schema)
    inputs     = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
    input_len  = inputs["input_ids"].shape[1]

    ft_model.config.pad_token_id = tokenizer.eos_token_id
    out = ft_model.generate(
        **inputs,
        max_new_tokens=128,
        num_beams=5,
        early_stopping=True,
        do_sample=False,
    )
    gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
    sql = re.split(r"(###|\n\s*\n|```)", gen_text, maxsplit=1)[0]
    sql = sql.split(";", 1)[0]
    return sql.strip()


In [ ]:
sample = spider_val[835]
schema = get_schema_string(sample["db_id"])
print(sample["question"])
print(sample['query'])
print(generate_sql(sample["question"], schema))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Count the number of different nationalities of conductors.
SELECT count(DISTINCT Nationality) FROM conductor
SELECT count(DISTINCT nationality) FROM conductor


# Defining Evaluation Metrics

In [ ]:
def exact_match(pred, gold):
    return pred.strip().lower() == gold.strip().lower()




from sqlglot import parse_one, expressions

def flatten_ast(node):
    """
    Recursively collect all node‐type names and literal values as lowercase strings.
    """
    out = set()

    def walk(n):
        # record the AST node type
        out.add(type(n).__name__.lower())

        # record any literal (e.g. identifiers, strings, numbers)
        if hasattr(n, "this") and isinstance(n.this, (str, int, float)):
            out.add(str(n.this).lower())

        # recurse into child expressions
        for arg in n.args.values():
            if isinstance(arg, list):
                for child in arg:
                    if isinstance(child, expressions.Expression):
                        walk(child)
            elif isinstance(arg, expressions.Expression):
                walk(arg)

    walk(node)
    return out

def component_match(pred_sql, gold_sql):
    try:
        pred_ast = parse_one(pred_sql)
        gold_ast = parse_one(gold_sql)
    except Exception:
        return 0.0

    pred_set = flatten_ast(pred_ast)
    gold_set = flatten_ast(gold_ast)
    if not gold_set:
        return 0.0
    return len(pred_set & gold_set) / len(gold_set)




import sqlite3, pandas as pd, numpy as np
from tqdm import tqdm

def run_query(sql: str, db_path: Path):
    """Return query result as a sorted list of tuples (order-independent)."""
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query(sql, conn)
        # sort rows + cols for order-invariant comparison
        return tuple(map(tuple, df.sort_index(axis=1).sort_values(list(df.columns)).to_numpy()))
    except Exception as e:
        # any failure counts as wrong
        return f"ERROR-{e}"

def execution_accuracy(dataset):
    """Compute Spider-style Execution Accuracy on a HF split (e.g. validation)."""
    correct = 0
    for ex in tqdm(dataset, desc="Evaluating"):
        db_id   = ex["db_id"]
        schema  = get_schema_string(db_id)
        pred_sql= generate_sql(ex["question"], schema)

        db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
        gold    = run_query(ex["query"],     db_file)
        pred    = run_query(pred_sql,        db_file)

        if gold == pred:
            correct += 1

    return correct / len(dataset)

# Eval on 20% Split

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [20:43<00:00,  6.04s/it]



Exact Match        : 0.16019417475728157
Component Match    : 0.9246873988669715
Execution Accuracy : 0.6019417475728155
Avg. Latency  (s)  : 6.016378070708761
95% Latency  (s)   : 14.137315094249743
GPU Mem Peak       : 31.990704128 GB


# Introducing 5-shot prompt (Random Sample and MiniLM) on the Qlora Adapters

In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 0 ▪ global seed (reproducible shuffles, torch, numpy, random, etc.)
# ──────────────────────────────────────────────────────────────────────────
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ──────────────────────────────────────────────────────────────────────────
# 1 ▪ zero-shot prompt builder (no demos)
# ──────────────────────────────────────────────────────────────────────────
def build_prompt_zero(nl_question: str, target_schema: str) -> str:
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given the database schema and the "
        "question, return ONE valid SQL statement — output ONLY the SQL.\n\n"
        f"### Database Schema:\n{target_schema}\n\n"
        f"### Question:\n{nl_question}\n\n"
        "### SQL:\n"
    )

# ──────────────────────────────────────────────────────────────────────────
# 2 ▪ random-5-shot builder
# ──────────────────────────────────────────────────────────────────────────
NUM_SHOTS = 5
spider_train = load_dataset("spider")["train"]
DEMO_SET = [
    {**ex, "schema_str": get_schema_string(ex["db_id"])}
    for ex in spider_train.shuffle(seed=SEED).select(range(NUM_SHOTS))
]

def build_prompt_random5(nl_question: str, target_schema: str) -> str:
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]

    for i, ex in enumerate(DEMO_SET, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 3 ▪ retrieved-5-shot builder (SBERT nearest neighbours)
# ──────────────────────────────────────────────────────────────────────────
from sentence_transformers import SentenceTransformer, util
embedder   = SentenceTransformer("all-MiniLM-L6-v2")
train_emb  = embedder.encode(spider_train["question"], convert_to_tensor=True)

def get_k_shots(nl_question: str, k: int = 5):
    q_emb  = embedder.encode(nl_question, convert_to_tensor=True)
    hits   = util.semantic_search(q_emb, train_emb, top_k=k)[0]
    demos  = []
    for h in hits:
        ex = spider_train[int(h["corpus_id"])]
        demos.append({
            "question"   : ex["question"],
            "query"      : ex["query"],
            "schema_str" : get_schema_string(ex["db_id"]),
        })
    return demos

def build_prompt_retrieved5(nl_question: str, target_schema: str) -> str:
    demos = get_k_shots(nl_question, 5)
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]

    for i, ex in enumerate(demos, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
            "### End\n",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 4 ▪ shared generate_sql that accepts a *builder* argument
# ──────────────────────────────────────────────────────────────────────────
import re

def make_generator(builder_fn):
    def _gen(nl_question: str, schema: str) -> str:
        prompt = builder_fn(nl_question, schema)
        inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
        input_len = inputs["input_ids"].shape[1]

        ft_model.config.pad_token_id = tokenizer.eos_token_id
        out = ft_model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=5,
            early_stopping=True,
            do_sample=False,
        )
        gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
        sql = re.split(r"(###|\n\s*\n|```)", gen_text, 1)[0]
        return sql.split(";", 1)[0].strip()
    return _gen

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# ──────────────────────────────────────────────────────────────────────────
# 5 ▪ run the usual 20 % dev evaluation (exact same loop you used before)
# ──────────────────────────────────────────────────────────────────────────
# … sample_val creation …
# … em_scores, cm_scores, ex_scores, times …
# loop exactly as before – now uses the new generate_sql


# Evaluation for 5-shot (Random) prompt for Qlora Adapters

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 1          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [23:37<00:00,  6.88s/it]



Exact Match        : 0.1650485436893204
Component Match    : 0.9382870428355189
Execution Accuracy : 0.6067961165048543
Avg. Latency  (s)  : 6.8623649788835355
95% Latency  (s)   : 15.401736658000118
GPU Mem Peak       : 32.782470656 GB


# Evaluation for 5-shot (all-MiniLM) prompt for Qlora Adapters

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [53:16<00:00, 15.52s/it]



Exact Match        : 0.18932038834951456
Component Match    : 0.9535244202071526
Execution Accuracy : 0.6019417475728155
Avg. Latency  (s)  : 15.493059734155315
95% Latency  (s)   : 16.152427036998688
GPU Mem Peak       : 36.770135552 GB
